In [1]:
# user-configurable
source = 'nj'
route = '119'
stop = '30189'
# db_server='192.168.1.181'
db_server='127.0.0.1'


period = 'today'

In [2]:
# import libraries
import datetime, time, sys
from operator import itemgetter
import pandas as pd

In [3]:
# import NJT API app libraries
import reportcard.lib.BusAPI as BusAPI

In [4]:
# work with bus position log
import reportcard.lib.BusRouteLogsDB as BusRouteLogsDB
db = BusRouteLogsDB.MySQL('buses', 'buswatcher', 'njtransit', db_server, route)
conn = db.conn
table_name = 'routelog_' + route

In [10]:
if period == "today":
    query = ('SELECT * FROM %s WHERE (rt=%s AND DATE(`timestamp`)=CURDATE() ) ORDER BY run, timestamp;' % (table_name, route))
elif period == "yesterday":
    query = ('SELECT * FROM %s WHERE (rt=%s AND stop_id= %s AND (timestamp >= CURDATE() - INTERVAL 1 DAY AND timestamp < CURDATE())) ORDER BY run, timestamp;' % (table_name, route))
else:
    raise RuntimeError('Bad request sucker!')

In [60]:
# just get a specific date out
query = ('SELECT * FROM routelog_119 WHERE (rt=119 AND datediff(timestamp, "20180926") = 0 ) ORDER BY run, timestamp;')

In [61]:
# get a single days positions table into a dataframe
df = pd.read_sql_query(query, conn)
df

,pkey,lat,lon,bid,cars,consist,d,dip,dn,fs,...,pdRtpiFeedName,pid,rt,rtRtpiFeedName,rtdd,rtpiFeedName,run,wid1,wid2,timestamp
0,22723,40.757450,-73.992415,8136,,,South East Bound,1727,ESE,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,...,,1860,119,,119,,10,010,0119,2018-09-26 14:47:01.066735
1,22730,40.757450,-73.992415,8136,,,South East Bound,1727,ESE,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,...,,1860,119,,119,,10,010,0119,2018-09-26 14:48:00.617549
2,22737,40.757450,-73.992415,8136,,,South East Bound,1727,ESE,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,...,,1860,119,,119,,10,010,0119,2018-09-26 14:49:01.000716
3,22744,40.757450,-73.992415,8136,,,South East Bound,1727,ESE,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,...,,1860,119,,119,,10,010,0119,2018-09-26 14:50:01.511973
4,22751,40.757450,-73.992415,8136,,,South East Bound,1727,ESE,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,...,,1860,119,,119,,10,010,0119,2018-09-26 14:51:01.199158
5,22758,40.758018,-73.994141,8136,,,North West Bound,2268,WNW,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,...,,1860,119,,119,,10,010,0119,2018-09-26 14:52:01.635052
6,22765,40.757778,-73.995888,8136,,,North West Bound,2768,WNW,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,...,,1860,119,,119,,10,010,0119,2018-09-26 14:53:01.364352
7,22772,40.758739,-73.998657,8136,,,North West Bound,3616,WNW,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,...,,1860,119,,119,,10,010,0119,2018-09-26 14:54:00.574886
8,22779,40.760502,-74.003342,8136,,,North West Bound,5069,WNW,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,...,,1860,119,,119,,10,010,0119,2018-09-26 14:55:00.875398
9,22787,40.761089,-74.004768,8136,,,North West Bound,5520,WNW,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,...,,1860,119,,119,,10,010,0119,2018-09-26 14:56:00.829120


In [62]:
# split final approach history (sorted by timestamp) at each change in vehicle_id outputs a list of dfs -- per https://stackoverflow.com/questions/41144231/python-how-to-split-pandas-dataframe-into-subsets-based-on-the-value-in-the-fir

run_dfs = [g for i, g in df.groupby(df['run'].ne(df['run'].shift()).cumsum())]
run_dfs

[     pkey        lat        lon   bid cars consist                 d    dip  \
 0   22723  40.757450 -73.992415  8136               South East Bound   1727   
 1   22730  40.757450 -73.992415  8136               South East Bound   1727   
 2   22737  40.757450 -73.992415  8136               South East Bound   1727   
 3   22744  40.757450 -73.992415  8136               South East Bound   1727   
 4   22751  40.757450 -73.992415  8136               South East Bound   1727   
 5   22758  40.758018 -73.994141  8136               North West Bound   2268   
 6   22765  40.757778 -73.995888  8136               North West Bound   2768   
 7   22772  40.758739 -73.998657  8136               North West Bound   3616   
 8   22779  40.760502 -74.003342  8136               North West Bound   5069   
 9   22787  40.761089 -74.004768  8136               North West Bound   5520   
 10  22795  40.761089 -74.004768  8136               North West Bound   5520   
 11  22803  40.761089 -74.004768  8136  

In [63]:
len(run_dfs)

22

In [64]:
results = pd.DataFrame()

for run in run_dfs:
    start_report = run.head(1)
    end_report = run.tail(1)
    results = results.append(run.head(1))
    results = results.append(run.tail(1))
    

In [65]:
results

,pkey,lat,lon,bid,cars,consist,d,dip,dn,fs,...,pdRtpiFeedName,pid,rt,rtRtpiFeedName,rtdd,rtpiFeedName,run,wid1,wid2,timestamp
0,22723,40.757450,-73.992415,8136,,,South East Bound,1727,ESE,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,...,,1860,119,,119,,10,010,0119,2018-09-26 14:47:01.066735
49,23094,40.731197,-74.068062,8136,,,South West Bound,35357,SW,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,...,,1860,119,,119,,10,010,0119,2018-09-26 15:36:01.170031
50,21734,40.732908,-74.061933,8101,,,West Bound,300,W,119 NEW YORK,...,,99396,119,,119,,12,012,0010,2018-09-26 08:54:00.674345
125,23098,40.765179,-74.023308,8274,,,North East Bound,58479,NE,119 NEW YORK,...,,38040,119,,119,,12,012,0119,2018-09-26 15:36:01.170031
126,20948,40.742520,-74.051804,8121,,,South West Bound,26099,SW,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,...,,1860,119,,119,,901,901,0119,2018-09-26 07:13:01.165369
198,21778,40.726487,-74.071362,8121,,,North East Bound,34422,NNE,119 NEW YORK,...,,38040,119,,119,,901,901,0119,2018-09-26 08:58:01.368266
199,20953,40.689637,-74.105580,8123,,,South West Bound,54144,SSW,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,...,,1860,119,,119,,902,902,0119,2018-09-26 07:13:01.165369
428,22407,40.660843,-74.127445,8123,,,South West Bound,66479,SSW,119 JERSEY CITY VIA CENTRAL BAYOONNE VIA JFK BLVD,...,,1860,119,,119,,902,902,0119,2018-09-26 13:48:01.444798
429,21023,40.653301,-74.134941,8124,,,North East Bound,1848,NNE,119 NEW YORK,...,,38040,119,,119,,903,903,0119,2018-09-26 07:21:00.979869
494,21782,40.761589,-74.026581,8124,,,North East Bound,56880,NNE,119 NEW YORK,...,,38040,119,,119,,903,903,0119,2018-09-26 08:58:01.368266


In [66]:
results.to_csv('119_run_report.csv')